In [1]:
import numpy as np

import pandas as pd
import sys, os, time
from scipy.stats import norm
import matplotlib.pyplot as plt

%matplotlib inline

sys.path.append('../')

from src.data_structures import FactorGraph, PriorityQueue
from src.sampling_algorithms.factor_components import (gaussian_bounce, chain_bounce_fn, 
                                                       gaussian_grad_potential_fn, 
                                                       gaussian_chain_grad_potential_fn,
                                                       logistic, 
                                                       generate_logistic_bounce, 
                                                       alias_sample, 
                                                       lambda_r, 
                                                       grad_logistic, 
                                                       lambda_bound
                                                      )
from src.utils import (interp, get_xtv, get_first_moment, get_second_moment, get_var)
from src.sampling_algorithms import LocalBPS
from src.plots.arrow_plot import arrow_plot
from arviz.stats import ess

from matplotlib import rc
rc('text', usetex=False)

In [2]:
np.random.seed(0)
d = 2
T = 5
N = 100
alpha = 0.5

In [3]:
mu0 = np.array([0. for _ in range(d)])
sig0 = np.diag([1. for _ in range(d)])
mu1 = np.array([0. for _ in range(d)])
sig1 = sig0 + sig0
sig10 = sig0

mus = [np.array([0. for _ in range(d)]) for _ in range(T-1)]
sigs = []     
event_samplers = []
grad_fns = []

for t in range(T):
    if t == 0:
        sigs.append(2*sig0)
        event_samplers.append(chain_bounce_fn(mu1, mu0, sig1, sig0, sig10))
        grad_fns.append(gaussian_chain_grad_potential_fn(mu1, mu0, sig1, sig10, sig0))
    else:
        sigs.append(sigs[t-1]+sig0)
        event_samplers.append(chain_bounce_fn(mu1, mu0, sigs[t], sigs[t-1], sigs[t-1]))
        grad_fns.append(gaussian_chain_grad_potential_fn(mu1, mu0, sigs[t], sigs[t-1], sigs[t-1]))
    
bounce_fns = [gaussian_bounce(mu0, sig0)] + event_samplers

grad_factor_potential_fns = [gaussian_grad_potential_fn(mu0, sig0)] + grad_fns


factor_indices = [[i for i in range(d)]] + \
                 [[d*i +j for j in range(d)]+[d + d*i +j for j in range(d)] for i in range(T)] 

factor_potential_fns = [lambda x: x for _ in grad_factor_potential_fns]

nodes = list(set(n for f in factor_indices for n in f ))


graph = FactorGraph(dim_x=len(nodes),
                  factor_indices=factor_indices,
                  factor_potential_fns=factor_potential_fns,
                  grad_factor_potential_fns=grad_factor_potential_fns)

In [4]:
factor_indices

[[0, 1],
 [0, 1, 2, 3],
 [2, 3, 4, 5],
 [4, 5, 6, 7],
 [6, 7, 8, 9],
 [8, 9, 10, 11]]

In [5]:
np.random.seed(0)
init_x = np.random.randn(len(nodes))
init_v = np.random.randn(len(nodes))
local_bps = LocalBPS(init_x = init_x,
         init_v = init_v,
         factor_graph = graph,
         bounce_fns=bounce_fns,
         refresh_rate=0.001)

In [6]:
start = time.time()
nsim= 10**6
results = local_bps.simulate(nsim)
res = results
duration = time.time()-start
print(duration)

100%|██████████| 1000000/1000000 [01:57<00:00, 8523.98it/s]


119.81693625450134


In [7]:
burnin = 10**3
res[burnin:].shape

(999001, 3, 12)

In [11]:
coef_betas = []
for i in range(len(nodes)):
    x1,v1,t1=get_xtv(res[burnin:],i)
    m = get_first_moment(x1, v1, t1)
    v = get_second_moment(x1, v1, t1)-m**2
    print('M1: {0} M2: {1}'.format(m,np.sqrt(v)))
coef_betas = np.array(coef_betas)

M1: -0.001944710303532501 M2: 0.9950902285594222
M1: -0.0002239048014546897 M2: 0.9945789268185112
M1: 0.003038715048974063 M2: 1.3959126928204817
M1: -5.3929682404911026e-05 M2: 1.3947058548982643
M1: 0.002712072172535588 M2: 1.7020255880550634
M1: -0.0007649421511950726 M2: 1.7004677734332483
M1: -0.000868863042792006 M2: 1.963371799015955
M1: 0.0003109245378038754 M2: 1.9580046401107796
M1: -0.0005431358602712018 M2: 2.196575465507048
M1: 0.00161897097859196 M2: 2.188511938307283
M1: -0.0013351560774431477 M2: 2.4069348897758625
M1: -0.00033638757241877 M2: 2.400849760292328


In [9]:
coef_betas

array([], dtype=float64)

In [10]:
np.min(ps), np.max(ps)

NameError: name 'ps' is not defined

In [ ]:
for i in range(len(nodes)):
    x1,v1,t1=get_xtv(res,i)
    x = interp(x1,t1,v1, num_intervals=nsim*5)
    print('ESS {0}: {1}'.format('bulk',ess(x, method='bulk')/duration))
    print('ESS {0}: {1}'.format('tail',ess(x, method='tail')/duration))

In [ ]:
x1,v1,t1=get_xtv(res,0)
x2,v2,t2=get_xtv(res,1)

In [ ]:
x1,v1,t1=get_xtv(res,2)
x2,v2,t2=get_xtv(res,3)
burnin = 4000
plot_limit = 100000
fig_size = (10,10)
plt.rcParams["axes.edgecolor"] = "0.15"
plt.rcParams["axes.linewidth"]  = 1.25

font = {
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)


fig = plt.figure(figsize=fig_size,frameon =True)
left, bottom, width, height = 0.1, 0.1, 0.8, 0.8
ax = fig.add_axes([left, bottom, width, height])
ax.set_title("Local BPS")
ax.set_xlabel("$x_1$",fontsize='large', fontweight='bold')
ax.set_ylabel("$x_2$",fontsize='large', fontweight='bold')
arrow_plot(x1[burnin:burnin+plot_limit],x2[burnin:burnin+plot_limit], head_length=0.01,head_width=0.01)
fig.savefig('./local_bps.eps', format='eps', dpi=1200)



In [ ]:
x1,v1,t1=get_xtv(res[1000:],2)

In [ ]:
plt.plot(x1)

In [ ]:
np.mean(x1)

In [ ]:
np.std(x1)